<a href="https://colab.research.google.com/github/rmccormick-contractor/Hyperion-ASD-Visualization/blob/main/Hyperion_ASD_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#First time setup - Installs all required components
!pip install geemap
!pip install ipyleaflet
!pip install ipywidgets
!pip install ipympl
!pip install npm
!pip install gmaps
!pip install hvplot
!pip install geopy
!pip install pygeocoder
!pip install matplotlib
!pip uninstall tornado --yes
!pip install tornado

# RESTART KERNEL AFTER THIS CODE HAS BEEN RUN

In [ ]:
#Import dependencies
import pandas as pd
import numpy as np
import geopy
import geemap
import hvplot.pandas
import panel as pn
import holoviews as hv
import matplotlib as mpl
import matplotlib.pyplot as plt

from bokeh.models import SingleIntervalTicker, LinearAxis
from holoviews import opts
hv.extension('bokeh')
from pygeocoder import Geocoder
from IPython.display import display

#Cleanup data for mapping. (Map does not need spectrum data)
#Map_DF = pd.read_csv(r'GHISA-USA-EO-1-Hyperion-data-of-crops-1h_gaezs.csv')
#Map_DF = Map_DF.drop(Map_DF.iloc[:, 12:-2], axis=1)

In [ ]:
pn.extension(comms='colab')

Combined_Crop_Select = pn.widgets.Select(name='Crop:', options=['All', 'Alfalfa', 'Cotton', 'Corn', 'Rice', 'Soybean', 'Wheat', 'Winter Wheat'])
Combined_Crop_Select

Combined_Stage_Select = pn.widgets.Select(
    name='Growth Stage:', 
    options=['All', 'Emerge_VEarly', 'Early_Mid', 'Mature_Senesc', 'Late', 'Harvest', 'Critical']
)

Year_Range_Slider = pn.widgets.RangeSlider(
    name='Select Year(s)', start=2006, end=2015, step=1)

Month_Range_Slider = pn.widgets.RangeSlider(
    name='Select Month(s)', start=1, end=12, step=1)

Day_Range_Slider = pn.widgets.RangeSlider(
    name='Select Julian Day(s)', start=1, end=365, step=1)

AEZ_Global_Select = pn.widgets.IntSlider(name='AEZ (Global, 0 for all)', start=0, end=74)

Country_Select = pn.widgets.Select(name='Country:', options=['All', 'USA', 'Uzbekistan'])

State_Select = pn.widgets.Select(
    name='Study Area:', 
    options=['All', 'Arizona', 'California', 'Indiana', 'Kansas', 'Oklahoma', 'Galaba', 'South Dakota', 'Texas', 'U.Yusupova', 'Wisconsin']
)


Sensor_Select = pn.widgets.Select(name='Sensor:', options=['All', 'Hyperion', 'ASD Spectroradiometer'])


pn.Row(pn.WidgetBox(Sensor_Select, Combined_Crop_Select, Combined_Stage_Select), 
       pn.WidgetBox(Year_Range_Slider, Month_Range_Slider, Day_Range_Slider),
       pn.WidgetBox(AEZ_Global_Select, Country_Select, State_Select)
)

In [ ]:
# Open each data file and assign to a Data Frame

new_df = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/asd_long.csv', 
                     low_memory=False, index_col=False, na_values='NaN')
new_df2 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp1_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df2_1 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp2_1_long.csv',
                        low_memory=False, index_col=False, na_values='NaN')
new_df3 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp2_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df4 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp3_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df5 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp4_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')
new_df6 = pd.read_csv(r'https://raw.githubusercontent.com/rmccormick-contractor/Hyperion-ASD-Visualization/main/Croplands_DataViz/hyp5_long.csv',
                      low_memory=False, index_col=False, na_values='NaN')

# Concat all dataframes into one single
new_df = pd.concat([new_df, new_df2])
new_df = pd.concat([new_df, new_df2_1])
new_df = pd.concat([new_df, new_df3])
new_df = pd.concat([new_df, new_df4])
new_df = pd.concat([new_df, new_df5])
new_df = pd.concat([new_df, new_df6])

# Drop unnessary index column
new_df = new_df.drop(new_df.iloc[:, 0:0], axis=1)


# Filter on Crop & Growth Stage
if (Combined_Crop_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Crop'].values == Combined_Crop_Select.value)]
if (Combined_Stage_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Stage'].values == Combined_Stage_Select.value)]
   
# Filter based on Year range
new_df = new_df.loc[(new_df['Year'].values >= Year_Range_Slider.value[0])]
new_df = new_df.loc[(new_df['Year'].values <= Year_Range_Slider.value[1])]

# Filter based on Month range
new_df = new_df.loc[(new_df['Month'].values >= Month_Range_Slider.value[0])]
new_df = new_df.loc[(new_df['Month'].values <= Month_Range_Slider.value[1])]

# Filter based on Julian Day range
new_df = new_df.loc[(new_df['JulianDay'].values >= Day_Range_Slider.value[0])]
new_df = new_df.loc[(new_df['JulianDay'].values <= Day_Range_Slider.value[1])]

if (AEZ_Global_Select.value != 0):
    new_df["Global_AEZ"] = pd.to_numeric(new_df["Global_AEZ"], errors='coerce').astype('int')
    new_df = new_df.loc[(new_df['Global_AEZ'].values == AEZ_Global_Select.value)]
    
if (Sensor_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Sensor'].values == Sensor_Select.value)]
    
if (State_Select.value != 'All'):
    new_df = new_df.loc[(new_df['StudyArea'].values == State_Select.value)]
    
if (Country_Select.value != 'All'):
    new_df = new_df.loc[(new_df['Country'].values == Country_Select.value)]


test_df = new_df

#new_df = new_df.replace(0.0, 'NaN', regex=True)
#new_df = new_df.set_index('MasterID')
new_df = new_df.drop(columns=['Sensor', 'UniqueID', 'PlotID', 'StudyArea', 'Country', 'USA_AEZ', 'Global_AEZ', 'FID', 'Image', 'Month', 'Year', 'JulianDay', 'Long', 'Lat', 'Stage'])
#new_df = new_df.transpose()
#new_df["Reflectance"] = pd.to_numeric(new_df["Reflectance"], errors='coerce').astype('int')

#new_df = new_df.reset_index()

new_df = new_df.dropna()
#new_df = new_df.drop(new_df.iloc[0], axis=1)

#new_df = new_df.pivot_table(
#    index=["MasterID","Crop","Wavelength"],
#    values='Reflectance'
#)

new_df.to_csv('new_df_data')


if (new_df.empty):
    print("No data found for this query. Please expand search parameters.")

In [ ]:
hv.extension('bokeh')

reflectance_plot = new_df.hvplot.line(x = "Wavelength",y = "Reflectance", by="MasterID", legend=False).opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)

reflectance_plot

In [ ]:
mean_df = test_df
mean_df = mean_df.groupby(['Sensor', 'Wavelength', 'Crop'])['Reflectance'].mean()
mean_df = mean_df.dropna()

median_df = test_df
median_df = median_df.groupby(['Sensor', 'Wavelength', 'Crop'])['Reflectance'].median()
median_df = median_df.dropna()

mean_df
mean_df.to_csv('testing')

# mean_df.plot(x='Wavelength', y='Reflectance')

In [ ]:
Group_By_Select = pn.widgets.Select(name='Group By:', options=['All', 'Crop Type', 'Sensor'])
Group_By_Select

In [ ]:
hv.extension('bokeh')

if (Group_By_Select.value == 'Sensor'):
    mean_plot = mean_df.hvplot.line(x = "Wavelength",y = "Reflectance",by="Sensor",title='Average Reflectance by Sensor',ylabel='Reflectance (%)',xlabel='Wavelength (nm)').opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)
    median_plot = median_df.hvplot.line(x = "Wavelength",y = "Reflectance",by='Sensor',title='Median Reflectance by Crop Type',ylabel='Reflectance (%)',xlabel='Wavelength (nm)').opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)
    
if (Group_By_Select.value == 'Crop Type'):
    mean_plot = mean_df.hvplot.line(x = "Wavelength",y = "Reflectance",by='Crop',title='Average Reflectance by Crop Type',ylabel='Reflectance (%)',xlabel='Wavelength (nm)').opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)
    median_plot = median_df.hvplot.line(x = "Wavelength",y = "Reflectance",by='Crop',title='Median Reflectance by Crop Type',ylabel='Reflectance (%)',xlabel='Wavelength (nm)').opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)

if (Group_By_Select.value == 'All'):
    mean_plot = mean_df.hvplot.line(x = "Wavelength",y = "Reflectance",by=['Crop']).opts(fontsize={'title': 16, 'labels': 14, 'yticks': 12},xrotation=45, xticks=15)
    mean_plot = mean_df.hvplot.line(x = "Wavelength",y = "Reflectance",by=['Sensor'])

    
mean_plot + median_plot